In [6]:
import feedparser
import re

# Returns title and dictionary of word counts for an RSS feed
def getwordcounts(url):
    # Parse the feed
    d=feedparser.parse(url)
    wc={}
    # Loop over all the entries
    for e in d.entries:
        if 'summary' in e: summary=e.summary
        else: summary=e.description
        # Extract a list of words
        words=getwords(e.title+' '+summary)
        for word in words:
            wc.setdefault(word,0)
            wc[word]+=1
        import pdb; pdb.set_trace()
    return getattr(d.feed, 'title', 'Unknown title'), wc

In [7]:
def getwords(html):
    # Remove all the HTML tags
    txt=re.compile(r'<[^>]+>').sub('',html)
    # Split words by all non-alpha characters
    words=re.compile(r'[^A-Z^a-z]+').split(txt)
    # Convert to lowercase
    return [word.lower( ) for word in words if word!='']

In [8]:
apcount={}
wordcounts={}
for feedurl in file('resources/feedlist.txt','r'):
    title,wc=getwordcounts(feedurl)
    wordcounts[title]=wc
    for word,count in wc.items( ):
        apcount.setdefault(word,0)
        if count>1:
            apcount[word]+=1

In [10]:
d = feedparser.parse('http://feedparser.org/docs/examples/atom10.xml')

In [12]:
d = feedparser.parse('http://feeds.feedburner.com/37signals/beMH')